# Parse output files such as sTD-DFT-xTB and GFN2

Goal is to calculate descriptors and create a csv file containing this information for future prediction models

In [1]:
import pandas as pd
import numpy as np
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit import Chem
from rdkit.Chem import DataStructs
from numpy import linalg

### Parse GFN2-xTB output file for polarizability and dipole moment

Polarizability is in units of $au^3$ and dipole moment is in units of Debye

In [2]:
def parse_GFN2(filename, molecule, data):
    '''
    Parses through GFN2-xTB output files

    Parameters
    -----------
    filename: str
        path to output file
    molecule: str
        name of molecule
    dictionary: list
        list of descriptors to add to
    '''
    
    with open(filename, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        while line:
            if 'molecular dipole' in line:
                line = file.readline()
                line = file.readline()
                line = file.readline()
                    
                line_list = line.split()
                dipole_moment = float(line_list[-1])
                
            elif 'Mol. C8AA' in line:
                line = file.readline()
                line_list = line.split()
                
                polarizability = float(line_list[-1])

            line = file.readline()  
        line = file.readline()

        outputs = [dipole_moment, polarizability]
        
        data.extend(outputs)

        return data
    
    


### Parse sTD-DFT-xTB output files (*.stda) for energy levels and absorption spectrum

Extracts/computes the following descriptors:  
1. HOMO (eV)
2. HOMO-1 (eV)
3. LUMO (eV)
4. LUMO+1 (eV)
5. Fundamental Band Gap (eV) (LUMO-HOMO)
6. deltaHOMO - difference in energy between HOMO and HOMO-1 (eV)
7. deltaLUMO - difference in energy between LUMO and LUMO+1 (eV)
8. Optical Band Gap (eV) - energy of the first transition within the first 12 transition with an oscillator strength greater than 0.5
9. Max abs (nm) - transition with the strongest absorption
10. summed_oscs - Sum of oscillator strengths (unitless)
11. area_spectra - area under the absorption spectrum curve using trapzoidal rule integration
12. area_sim_solar_spectra - area under the curve of the simulated spectrum multiplied by the normalized solar spectrum
13. chemical_potential (eV): $(HOMO+LUMO)/2$
14. electrophilicity (eV): $chemical potential^2 / 2(LUMO-HOMO)$


In [3]:
def parse_sTDA(filename, molecule, data):
    
    with open(filename, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        oscs = []
        wavelength = []
        energyEV = []
        while line:
            if 'ordered frontier orbitals' in line:
                for x in range(11):
                    line = file.readline()
                line_list = line.split()
                HOMOminus1 = float(line_list[1])
                
                line = file.readline()
                line_list = line.split()
                HOMO = float(line_list[1])
                
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                LUMO = float(line_list[1])
                line = file.readline()
                line_list = line.split()
                LUMOplus1 = float(line_list[1])

                deltaHOMO = abs(HOMOminus1 - HOMO)
                deltaLUMO = abs(LUMO - LUMOplus1)
                fundbg = abs(HOMO-LUMO)

            elif 'excitation energies, transition moments and TDA amplitudes' in line:
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                while line != '\n':
                    line_list = line.split()
                    oscs.append(float(line_list[3]))
                    wavelength.append(float(line_list[2]))
                    energyEV.append(float(line_list[1]))
                    line = file.readline()

            line = file.readline()  
        line = file.readline()
        
    chemical_potential = (HOMO + LUMO)/2
    hardness =  LUMO - HOMO
    # https://xtb-docs.readthedocs.io/en/latest/sp.html#global-electrophilicity-index
    electrophilicity = chemical_potential**2 / 2*hardness
   
    if len(oscs) != 0:
        summed_oscs = np.sum(oscs)
        
        highest_oscs = 0.0
        opt_bg = round(energyEV[0], 2)
        
        # Opt bg is the energy of the first transition within the first 12 transition with an oscillator strength greater than 0.5 
        if len(oscs) < 12:
            for i in range(len(oscs)):
                if  oscs[i] > 0.5:
                    opt_bg = round(energyEV[i], 2)
                    break
        else:
            for x in range(12):
                if  oscs[x] > 0.5:
                    opt_bg = round(energyEV[x], 2)
                    break

        # max abs is the tallest peak in the spectrum
        for x in range(len(oscs)):
            if  oscs[x] > highest_oscs:
                    highest_oscs = oscs[x]
                    max_abs = wavelength[x]
                    
        # Creates full spectrum
        (spectraEV, spectraNM, spectraIntensity) = spectra(energyEV, oscs)
        
        # Calculates the area under the curve using trapz rule for integration
        area_spectra = np.trapz(spectraIntensity, spectraNM, dx=0.1, axis=- 1)
        
        # Calculates the area under the curve of the simulated spectrum multiplied by the normalized solar spectrum
        area_sim_solar_spectra = solar_integrated_desc(spectraNM, spectraIntensity)
        
        outputs = [HOMO, HOMOminus1, LUMO, LUMOplus1, fundbg, deltaHOMO, deltaLUMO, opt_bg, max_abs, summed_oscs, area_spectra, area_sim_solar_spectra, chemical_potential, electrophilicity]

        data.extend(outputs)

        return data
    
    else:
        print(filename)
        print('something is wrong')
    

### Calculate the integration of the absorption spectra using trapezoidal integration

Multiplies the molecule's absorption spectra by the normalized solar spectrum. This new curve is then integrated with trapezoidal integration and the area if the descriptor used. 

In [4]:
def spectra(etens, etoscs, low = 0.5, high = 10.0, resolution = 0.01, smear = 0.04):
    """
    Return arrays of the energies and intensities of a Lorentzian-blurred spectrum

    Parameters
    ----------
    etens: list
        list of transition energies in units of eV
    etoscs: list
        list of oscillator strengths
    low: float
        transition in eV to start spectrum at
    high: float
        transition in eV to end spectrum at
    resolution: float
        increments of eV for spectrum
    smear: float
        blurs intensities of peaks across 0.04 eV

    Returns
    -------
    Lists of the spectra in eV, nm, and their oscillator strengths
    """
    maxSlices = int((high - low) / resolution) + 1
    peaks = len(etens)

    spectraEV = []
    spectraNM = []
    spectraIntensity = []
    for i in range(0, maxSlices):
        energy = float(i * resolution + low) # units of eV
        wavelength = energy * 1239.84193 # convert eV to nm  
        intensity = 0.0

        for trans in range(0, peaks):
            this_smear = smear / 0.2 * (-0.046 * etoscs[trans] + 0.20)
            deltaE = etens[trans] - energy
            intensity = intensity + etoscs[trans] * this_smear**2 / (deltaE**2 + this_smear**2)

        spectraEV.append(energy)
        spectraNM.append(wavelength) 
        spectraIntensity.append(intensity)
        
    return spectraEV, spectraNM, spectraIntensity

def custom_round(x, base=5):
    return float(base * round(float(x)/base))

def solar_integrated_desc(spectraNM, spectraIntensity):
    
    wavelength15AM = [] #wavelength for 1.5 AM spectra
    normalized_irr_15AM = []
    
    solar = pd.read_csv('Solar_radiation_spectrum.csv', index_col = 'wavelength')
    new_spectrum_intensities = []
    
    # the 1.5AM solar spectra does not have constant increments of wavelengths
    for x in range(len(spectraNM)):
        
        if 280 <= spectraNM[x] < 400:
            int_wavelength = custom_round(spectraNM[x], 0.5)
        if 400 <= spectraNM[x] < 1700:
            int_wavelength = custom_round(spectraNM[x], 1)
        if 1700 <= spectraNM[x] < 1702:
            int_wavelength = custom_round(spectraNM[x], 2)
        if 1702 <= spectraNM[x] <=4000:
            int_wavelength = custom_round(spectraNM[x], 5)

        solar_intensity = solar.loc[int_wavelength][-1]
        
        new_spectrum_intensities.append(float(solar_intensity) * spectraIntensity[x])
        
    area_altered_spectra = np.trapz(new_spectrum_intensities, spectraNM, dx=0.1, axis=- 1)
    
    return area_altered_spectra

### Descriptor to calculate the overlap between an acceptor and donor absorption spectra

Multiplies the simulated absorption spectra of the donor and acceptors using the sTD-DFT-xTB stda files. When multiplied, the only peaks left are where there is absorption overlap. For an ideal OSC, we want the materials to absorb in different regions to maximize the number of photons absorbed from the sun. This overlap is integrated and the area is the descriptor used for the models. Theoretically, the smaller the area, the higher the PCE.

Simpson's integration was tried but sometimes led to a negative area. Trapezoidal integration fit our type of data better.

In [5]:
def overlap(acceptor, donor, data):
    
    with open(acceptor, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        oscs = []
        energyEV = []
        while line:
            if 'excitation energies, transition moments and TDA amplitudes' in line:
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                while line != '\n':
                    line_list = line.split()
                    oscs.append(float(line_list[3]))
                    energyEV.append(float(line_list[1]))
                    line = file.readline()
            line = file.readline()  
        line = file.readline()
    
        # Creates full spectrum
        (acc_spectraEV, acc_spectraNM, acc_spectraIntensity) = spectra(energyEV, oscs)
        
    with open(donor, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        oscs = []
        energyEV = []
        while line:
            if 'excitation energies, transition moments and TDA amplitudes' in line:
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                while line != '\n':
                    line_list = line.split()
                    oscs.append(float(line_list[3]))
                    energyEV.append(float(line_list[1]))
                    line = file.readline()
            line = file.readline()  
        line = file.readline()
        
   
        # Creates full spectrum
        (don_spectraEV,  don_spectraNM, don_spectraIntensity) = spectra(energyEV, oscs)
        
    overlapped_spectra_intensities = [don_spectraIntensity[i] * acc_spectraIntensity[i] for i in range(len(don_spectraIntensity))]

    
    area_altered_spectra = np.trapz(overlapped_spectra_intensities, don_spectraNM, dx=0.1, axis=- 1)

    outputs = [area_altered_spectra]
    
    data.extend(outputs)
        
    return data

### Calculates the number of atoms in the conjugation path

In [6]:
def getPiSystemSize(mol):
    mol = AllChem.RemoveHs(mol)
    AllChem.Kekulize(mol)
    pi_systems = [pi_system(mol,x.GetIdx(),[x.GetIdx()]) for x in mol.GetAtoms()]
    largest_pi_system = max(pi_systems, key=lambda coll: len(coll))
    pi_system_size = len(largest_pi_system)
    return pi_system_size

def pi_system(mol, current, seen):
    atom = mol.GetAtomWithIdx(current)
    for neighbor in atom.GetNeighbors():
        if (neighbor.GetIdx() not in seen) and (mol.GetBondBetweenAtoms(atom.GetIdx(),neighbor.GetIdx()).GetIsConjugated() or mol.GetBondBetweenAtoms(atom.GetIdx(),neighbor.GetIdx()).GetBondTypeAsDouble() > 1):
            seen.append(neighbor.GetIdx())
            pi_system(mol,neighbor.GetIdx(),seen)
    return seen

def pi_sys_size(filename, molecule, data):
    mol = AllChem.MolFromMolFile(filename)
    pi_size = getPiSystemSize(mol)
    
    outputs = [pi_size]

    data.extend(outputs)

    return data

### Calculate planarity of pi system

https://github.com/rdkit/rdkit/tree/master/Contrib/PBF

Modified from 
J. Chem. Inf. Model. 2012, 52, 10, 2516–2525
https://pubs.acs.org/doi/10.1021/ci300293f

In [7]:
def GetBestFitPlane(pts, weights=None):
    # number of atoms
    wSum = len(pts)
    # sets the origin as the sum of the coordinates for x, y, and z
    origin = np.sum(pts, 0)
    # finds the average of each coordinate and sets as the origin
    origin /= wSum

    # initiates blank coordinates
    sumXX = 0
    sumXY = 0
    sumXZ = 0
    sumYY = 0
    sumYZ = 0
    sumZZ = 0
    sums = np.zeros((3, 3), np.double)
    
    # finds the distance of each point to origin
    for pt in pts:
        # finds the distance of each point to origin
        dp = pt - origin
        
        # sets the 3x3 matrix
        for i in range(3):
            sums[i, i] += dp[i] * dp[i]
            for j in range(i + 1, 3):
                sums[i, j] += dp[i] * dp[j]
                sums[j, i] += dp[i] * dp[j]
    # Averages each number in matrix by the total number of atoms
    sums /= wSum
    
    # Finds the eigenvalues and eigenvectors 
    vals, vects = linalg.eigh(sums)

    # gives indices sorted from smallest to largest
    order = np.argsort(vals)
    
    # smallest eigenvector
    normal = vects[:, order[0]]    
    
    # sets plane coordinates
    plane = np.zeros((4, ), np.double)
    plane[:3] = normal
    plane[3] = -1 * normal.dot(origin)
    
    return plane


def PBFRD(mol, largest_pi_system, confId=-1):
    conf = mol.GetConformer(confId)
    if not conf.Is3D():
        return 0
    
    pts = np.array([list(conf.GetAtomPosition(x)) for x in largest_pi_system])
    plane = GetBestFitPlane(pts)
    
    #distance to point
    denom = np.dot(plane[:3], plane[:3])
    denom = denom**0.5
    # add up the distance from the plane for each point:
    res = 0.0
    for pt in pts:
        res += np.abs(pt.dot(plane[:3]) + plane[3])
        
    res /= denom
    res /= len(pts)
    
    # higher the number, the less planar it is
    return res

def planarity (filename, data):
    mol = Chem.MolFromMolFile(filename)
    mol = Chem.RemoveHs(mol)
    Chem.Kekulize(mol)
    pi_systems = [pi_system(mol,x.GetIdx(),[x.GetIdx()]) for x in mol.GetAtoms()]
    largest_pi_system = max(pi_systems, key=lambda coll: len(coll))

    planarity = PBFRD(mol, largest_pi_system)
    
    outputs = [planarity]
    data.extend(outputs)
    
    return data

### RDKit descriptors

1. num_rot_bonds = number of rotatable bonds
2. MolLogP (Wildman and Crippen JCICS 39:868-73 (1999))
3. TPSA (J. Med. Chem. 43:3714-7, (2000))
4. NumHAcceptors
5. NumHDonors

In [8]:
def rdkit_descriptors(filename, data):
    mol = Chem.MolFromMolFile(filename)
    num_rot_bonds = Descriptors.NumRotatableBonds(mol)
    MolLogP = Descriptors.MolLogP(mol)
    TPSA = Descriptors.TPSA(mol)
    NumHAcceptors = Descriptors.NumHAcceptors(mol)
    NumHDonors = Descriptors.NumHDonors(mol)
    RingCount = Descriptors.RingCount(mol)

    outputs = [num_rot_bonds, MolLogP, TPSA, NumHAcceptors, NumHDonors, RingCount]
    
    data.extend(outputs)
    return data

## Morgan Fingerprints

### Morgan Fingerprint as Bit Vect

In [9]:
def morgan_fp(filename, data):
    mol = Chem.MolFromMolFile(filename)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    outputs = list(fp)
    
    data.extend(outputs)
    return data

### Morgan Fingerprint as counts

In [10]:
'''filename = '../Calculations/acceptors/GFN2/2FIFIC.mol'
mol = Chem.MolFromMolFile(filename)
fp3 = AllChem.GetHashedMorganFingerprint(mol, 2, nBits=2048)
array = np.zeros((0,), dtype=np.int8)
DataStructs.ConvertToNumpyArray(fp3, array)
#print(array.nonzero())'''

def numpy_2_fp(array):
    fp = DataStructs.cDataStructs.UIntSparseIntVect(len(array))
    for ix, value in enumerate(array):
        fp[ix] = int(value)
    return fp
data = []
def morgan_fp_counts(filename, data):
    mol = Chem.MolFromMolFile(filename)
    fp3 = AllChem.GetHashedMorganFingerprint(mol, 2, nBits=2048)
    array = np.zeros((0,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp3, array)
    
    fp4 = numpy_2_fp(array)
    
    outputs = list(fp4)
    
    data.extend(outputs)
    return data



### Solvation energy

https://xtb-docs.readthedocs.io/en/latest/gbsa.html

In [11]:
def solvation(filename, data):
    with open(filename, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        while line:
            if '-> Gsolv' in line:
                line_list = line.split()
                solvation_energy = float(line_list[3])
                break
                
            line = file.readline()  
        line = file.readline()

    outputs = [solvation_energy]
        
    data.extend(outputs)

    return data

### Offsets in energy levels

1. HOMO offset between donor and acceptor
2. LUMO offset between donor and acceptor
3. difference in energy between donor's HOMO and acceptor's LUMO (thought to be enegry of CT state)

In [12]:
def energy_offsets(acceptor, donor, data):
    with open(acceptor, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        while line:
            if 'ordered frontier orbitals' in line:
                for x in range(12):
                    line = file.readline()
                line_list = line.split()
                acc_HOMO = float(line_list[1])
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                acc_LUMO = float(line_list[1])
                break
            line = file.readline()  
        line = file.readline()
        
    with open(donor, 'r', encoding = 'utf-8') as file:
        line = file.readline()
        while line:
            if 'ordered frontier orbitals' in line:
                for x in range(12):
                    line = file.readline()
                line_list = line.split()
                don_HOMO = float(line_list[1])
                line = file.readline()
                line = file.readline()
                line_list = line.split()
                don_LUMO = float(line_list[1])
                break
            line = file.readline()  
        line = file.readline()
        
    HOMO_offset = don_HOMO - acc_HOMO
    LUMO_offset = don_LUMO - acc_LUMO
    DonHOMO_accLUMO = acc_LUMO - don_HOMO
                    
    outputs = [HOMO_offset, LUMO_offset, DonHOMO_accLUMO]
    data.extend(outputs)

    return data

# Create dataframe with all descriptors

Make code to create new dataframe based on the experimental pairs. Then can add the HOMO offset between A and D, etc. 


In [13]:
experimental = pd.read_csv('../filtered_experimental_PCE_params.csv')
experimental

,Reported Acceptor,acceptor filename shortened chains,donor filename shortened chains,Reported Donor,Reference,Voc (V),Jsc (mA/cm^2),FF (%),PCE (%),Class,acceptor type,acceptor class,core fused ring size,donor
0,2FIFIC,2FIFIC,PBDB-TF,PBDB-TF,Zhang_preparation_2020,0.800,22.40,61.50,11.34,NaN,sm,fused,5.0,NaN
1,2T2Se-F,2T2Se-F,PBDB-TF,PBDB-TF,Ding_selenophene_2022,0.875,20.63,66.90,12.05,polymer/SM,sm,unfused,1.0,NaN
2,3TT-OCIC,3TT-OCIC,PTB7-Th,PTB7-Th,Gao_achieving_2019,0.680,26.49,69.00,12.43,NaN,sm,fused,8.0,NaN
3,4T-3,4T-1,D18,D18,Zhou_high-efficiency_2021,0.930,18.28,70.97,12.04,NaN,sm,unfused,1.0,NaN
4,4T-3,4T-1,PBDB-T,PBDB-T,Zhou_high-efficiency_2021,0.810,17.27,72.45,10.15,NaN,sm,unfused,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,ZITI-N,ZITI-N,PTQ10,PTQ10,Zhang_accurate_2020,0.967,12.03,60.68,7.06,NaN,NaN,NaN,NaN,NaN
997,ZITI-N,ZITI-N,J71,J71,Zhang_revealing_2019,0.876,21.78,72.00,13.68,NaN,NaN,NaN,NaN,NaN
998,ZITI-S,ZITI-S,J71,J71,Zhang_revealing_2019,0.811,17.39,64.62,9.12,NaN,NaN,NaN,NaN,NaN
999,ZITI-S,ZITI-S,PTQ10,PTQ10,Zhang_accurate_2020,0.960,16.94,65.75,10.69,NaN,NaN,NaN,NaN,NaN


In [15]:
column_names = ['acceptor', 'donor', 'reference', 'Voc', 'Jsc', 'FF', 'PCE', 'A-HOMO', 'A-HOMOminus1', 'A-LUMO', 'A-LUMOplus1', 'A-fundbg', 'A-deltaHOMO', 'A-deltaLUMO', 'A-opt_bg', 'A-max_abs', 'A-summed_oscs', 'A-area_spectra', 'A-area_sim_solar_spectra', 'A-chemical_potential', 'A-electrophilicity', 'A-pi_sys_size', 'A-num_rot_bonds', 'A-MolLogP', 'A-TPSA', 'A-NumHAcceptors', 'A-NumHDonors', 'A-RingCount','A-planarity','A-dipole_moment', 'A-polarizability', 'A-SolvationEnergy_water', 'A-SolvationEnergy_hexane', 'D-HOMO', 'D-HOMOminus1', 'D-LUMO', 'D-LUMOplus1', 'D-fundbg', 'D-deltaHOMO', 'D-deltaLUMO', 'D-opt_bg', 'D-max_abs', 'D-summed_oscs', 'D-area_spectra', 'D-area_sim_solar_spectra', 'D-chemical_potential', 'D-electrophilicity', 'D-pi_sys_size', 'D-num_rot_bonds', 'D-MolLogP', 'D-TPSA', 'D-NumHAcceptors', 'D-NumHDonors', 'D-RingCount',  'D-planarity','D-dipole_moment', 'D-polarizability', 'D-SolvationEnergy_water', 'D-SolvationEnergy_hexane', 'AD-overlap', 'AD-HOMOoffset', 'AD-LUMOoffset', 'DHOMO_ALUMO_offset']

# add column names for 2048 bit morgan fingerprints
for x in range(2048):
    col_name = 'A-ECFP_' + str(x)
    column_names.append(col_name)
for x in range(2048):
    col_name = 'D-ECFP_' + str(x)
    column_names.append(col_name)

df = pd.DataFrame(columns = column_names)

for x in range(len(experimental)):
    try:
    
        data = []

        acceptor = experimental.iloc[x][1]
        donor = experimental.iloc[x][2]
        reference = experimental.iloc[x][4]
        Voc = experimental.iloc[x][5]
        Jsc = experimental.iloc[x][6]
        FF = experimental.iloc[x][7]
        PCE = experimental.iloc[x][8]

        data.extend([acceptor, donor, reference, Voc, Jsc, FF, PCE])

        
        # filenames
        acc_stda = '../Calculations/acceptors/sTDDFT-xTB/' + acceptor + '.stda'
        acc_mol = '../Calculations/acceptors/GFN2/' + acceptor + '.mol'
        acc_GFN2 = '../Calculations/acceptors/GFN2/' + acceptor + '.out'
        acc_solv_water = '../Calculations/acceptors/xtb_solvation_water/' + acceptor + '.out'
        acc_solv_hexane = '../Calculations/acceptors/xtb_solvation_hexane/' + acceptor + '.out'
        don_stda = '../Calculations/donors/sTDDFT-xTB/' + donor + '.stda'
        don_mol = '../Calculations/donors/GFN2/' + donor + '.mol'
        don_GFN2 = '../Calculations/donors/GFN2/' + donor + '.out'
        don_solv_water = '../Calculations/donors/xtb_solvation_water/' + donor + '.out'
        don_solv_hexane = '../Calculations/donors/xtb_solvation_hexane/' + donor + '.out'



        # parse sTDDFT-xtb output files of acceptors
        parse_sTDA(acc_stda, acceptor, data) #HOMO, HOMOminus1, LUMO, LUMOplus1, fundbg, deltaHOMO, deltaLUMO, opt_bg, max_abs, summed_oscs, area_spectra, area_sim_solar_spectra, chemical_potential, electrophilicity

        # parse GFN2-xtb files for acceptor
        pi_sys_size(acc_mol, acceptor, data) #pi_size
        rdkit_descriptors(acc_mol, data) #num_rot_bonds, MolLogP, TPSA, NumHAcceptors, NumHDonors
        planarity(acc_mol, data) # planarity

        # calculate pi system size of acceptor
        parse_GFN2(acc_GFN2, acceptor, data) #dipole_moment, polarizability

        # calculate solvation free energy of acceptor in water
        solvation(acc_solv_water, data) #solvation_energy

        # calculate solvation free energy of acceptor in hexane
        solvation(acc_solv_hexane, data) #solvation_energy

        # parse sTDDFT-xtb output files of donors
        parse_sTDA(don_stda, donor, data) #HOMO, HOMOminus1, LUMO, LUMOplus1, fundbg, deltaHOMO, deltaLUMO, opt_bg, max_abs, summed_oscs, area_spectra, area_sim_solar_spectra, chemical_potential, electrophilicity

        # parse GFN2-xtb files for donor
        pi_sys_size(don_mol, donor, data) #pi_size
        rdkit_descriptors(don_mol, data) #num_rot_bonds, MolLogP, TPSA, NumHAcceptors, NumHDonors
        planarity(don_mol, data) # planarity

        # calculate pi system size of donor
        parse_GFN2(don_GFN2, donor, data) #dipole_moment, polarizability

        # calculate solvation free energy of donor in water
        solvation(don_solv_water, data) #solvation_energy

        # calculate solvation free energy of donor in hexane
        solvation(don_solv_hexane, data) #solvation_energy

        # calculate overlap between acceptor and donor
        overlap(acc_stda, don_stda, data) #area_altered_spectra

        # Energy offsets between donor and acceptor
        energy_offsets(acc_stda, don_stda, data)

        # For morgan fingerprint as bit vect
        '''
        # acceptor morgan fingerprint
        morgan_fp(acc_mol, data)

        # donor morgan fingerprint
        morgan_fp(don_mol, data)
        '''
        
        # For morgan fingerprint counts
        morgan_fp_counts(acc_mol, data)
        morgan_fp_counts(don_mol, data)
        

        df.loc[len(df.index)] = data
        
    except:
        print(acceptor)
        print(donor)
        continue
df

A-WSSe-Cl
PBDB-TF
CTSe-IC
J71


,acceptor,donor,reference,Voc,Jsc,FF,PCE,A-HOMO,A-HOMOminus1,A-LUMO,...,D-ECFP_2038,D-ECFP_2039,D-ECFP_2040,D-ECFP_2041,D-ECFP_2042,D-ECFP_2043,D-ECFP_2044,D-ECFP_2045,D-ECFP_2046,D-ECFP_2047
0,2FIFIC,PBDB-TF,Zhang_preparation_2020,0.800,22.40,61.50,11.34,-13.838,-14.653,-9.186,...,0,0,0,0,0,0,0,0,0,0
1,2T2Se-F,PBDB-TF,Ding_selenophene_2022,0.875,20.63,66.90,12.05,-13.067,-13.891,-9.590,...,0,0,0,0,0,0,0,0,0,0
2,3TT-OCIC,PTB7-Th,Gao_achieving_2019,0.680,26.49,69.00,12.43,-13.392,-14.307,-9.005,...,0,0,2,0,0,0,0,0,0,0
3,4T-1,D18,Zhou_high-efficiency_2021,0.930,18.28,70.97,12.04,-13.948,-14.850,-9.195,...,0,0,0,0,0,0,0,0,0,0
4,4T-1,PBDB-T,Zhou_high-efficiency_2021,0.810,17.27,72.45,10.15,-13.948,-14.850,-9.195,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,ZITI-N,PTQ10,Zhang_accurate_2020,0.967,12.03,60.68,7.06,-14.086,-14.504,-9.158,...,0,0,0,0,0,0,0,0,0,0
995,ZITI-N,J71,Zhang_revealing_2019,0.876,21.78,72.00,13.68,-14.086,-14.504,-9.158,...,0,0,0,0,0,0,0,0,0,0
996,ZITI-S,J71,Zhang_revealing_2019,0.811,17.39,64.62,9.12,-14.216,-14.785,-9.245,...,0,0,0,0,0,0,0,0,0,0
997,ZITI-S,PTQ10,Zhang_accurate_2020,0.960,16.94,65.75,10.69,-14.216,-14.785,-9.245,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df.to_csv('descriptors_with_fingerprints_counts.csv')

## Make csv with just donor properties

In [15]:
column_names = ['donor','D-HOMO', 'D-HOMOminus1', 'D-LUMO', 'D-LUMOplus1', 'D-fundbg', 'D-deltaHOMO', 'D-deltaLUMO', 'D-opt_bg', 'D-max_abs', 'D-summed_oscs', 'D-area_spectra', 'D-area_sim_solar_spectra', 'D-chemical_potential', 'D-electrophilicity', 'D-pi_sys_size', 'D-num_rot_bonds', 'D-MolLogP', 'D-TPSA', 'D-NumHAcceptors', 'D-NumHDonors', 'D-RingCount',  'D-planarity','D-dipole_moment', 'D-polarizability', 'D-SolvationEnergy_water', 'D-SolvationEnergy_hexane']

# add column names for 2048 bit morgan fingerprints
for x in range(2048):
    col_name = 'D-ECFP_' + str(x)
    column_names.append(col_name)

df = pd.DataFrame(columns = column_names)

donors = []
for x in range(len(experimental)):
    try:
        donor = experimental.iloc[x][2]
        
        if donor not in donors:
            donors.append(donor)
            data = []
            data.append(donor)

            # filenames
            don_stda = '../Calculations/donors/sTDDFT-xTB/' + donor + '.stda'
            don_mol = '../Calculations/donors/GFN2/' + donor + '.mol'
            don_GFN2 = '../Calculations/donors/GFN2/' + donor + '.out'
            don_solv_water = '../Calculations/donors/xtb_solvation_water/' + donor + '.out'
            don_solv_hexane = '../Calculations/donors/xtb_solvation_hexane/' + donor + '.out'


            # parse sTDDFT-xtb output files of donors
            parse_sTDA(don_stda, donor, data) #HOMO, HOMOminus1, LUMO, LUMOplus1, fundbg, deltaHOMO, deltaLUMO, opt_bg, max_abs, summed_oscs, area_spectra, area_sim_solar_spectra, chemical_potential, electrophilicity

            # parse GFN2-xtb files for donor
            pi_sys_size(don_mol, donor, data) #pi_size
            rdkit_descriptors(don_mol, data) #num_rot_bonds, MolLogP, TPSA, NumHAcceptors, NumHDonors
            planarity(don_mol, data) # planarity

            # calculate pi system size of donor
            parse_GFN2(don_GFN2, donor, data) #dipole_moment, polarizability

            # calculate solvation free energy of donor in water
            solvation(don_solv_water, data) #solvation_energy

            # calculate solvation free energy of donor in hexane
            solvation(don_solv_hexane, data) #solvation_energy

            morgan_fp_counts(don_mol, data)


            df.loc[len(df.index)] = data
        
    except:
        print(donor)
        continue
df

,donor,D-HOMO,D-HOMOminus1,D-LUMO,D-LUMOplus1,D-fundbg,D-deltaHOMO,D-deltaLUMO,D-opt_bg,D-max_abs,...,D-ECFP_2038,D-ECFP_2039,D-ECFP_2040,D-ECFP_2041,D-ECFP_2042,D-ECFP_2043,D-ECFP_2044,D-ECFP_2045,D-ECFP_2046,D-ECFP_2047
0,PBDB-TF,-13.374,-13.538,-8.177,-8.143,5.197,0.164,0.034,2.92,401.8,...,0,0,0,0,0,0,0,0,0,0
1,PTB7-Th,-13.046,-13.338,-8.152,-7.823,4.894,0.292,0.329,2.61,475.8,...,0,0,2,0,0,0,0,0,0,0
2,D18,-13.285,-13.396,-8.148,-8.005,5.137,0.111,0.143,3.03,409.8,...,0,0,0,0,0,0,0,0,0,0
3,PBDB-T,-13.225,-13.342,-8.184,-8.168,5.041,0.117,0.016,2.90,426.9,...,0,0,0,0,0,0,0,0,0,0
4,PBTIBDTT,-13.295,-13.405,-8.139,-8.018,5.156,0.110,0.121,3.00,413.4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,PBBSe-Cl,-13.302,-13.419,-8.360,-8.317,4.942,0.117,0.043,2.85,435.5,...,0,0,0,0,0,0,0,0,0,0
210,PBCT-2F,-13.369,-13.508,-8.220,-8.060,5.149,0.139,0.160,2.89,428.8,...,0,0,0,0,0,0,0,0,0,0
211,PE4,-13.215,-13.333,-8.289,-8.169,4.926,0.118,0.120,2.93,423.8,...,0,0,0,0,0,0,0,0,0,0
212,PvBDT-F,-13.343,-13.456,-7.944,-7.804,5.399,0.113,0.140,3.16,391.9,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df.to_csv('donor_properties.csv')

### Make a csv with the SMILES of each molecule

In [5]:
experimental = pd.read_csv('../filtered_experimental_PCE_params.csv')

column_names = ['molecule', 'AorD', 'reference', 'SMILES']
df_molecules = pd.DataFrame(columns = column_names)

acceptors = []
donors = []

for x in range(len(experimental)):
    acceptor = experimental.iloc[x][1]
    donor = experimental.iloc[x][2]
    reference = experimental.iloc[x][4]
    
    try:
        if acceptor not in acceptors:
            data = []
            acceptors.append(acceptor)
            data.extend([acceptor, 'A', reference])
            acc_mol = '../Calculations/acceptors/GFN2/' + acceptor + '.mol'
            mol = Chem.MolFromMolFile(acc_mol)
            smiles = Chem.MolToSmiles(mol)
            data.extend([smiles])

            df_molecules.loc[len(df_molecules.index)] = data

        if donor not in donors:
            data = []
            donors.append(donor)
            data.extend([donor, 'D', reference])
            don_mol = '../Calculations/donors/GFN2/' + donor + '.mol'
            mol = Chem.MolFromMolFile(don_mol)
            smiles = Chem.MolToSmiles(mol)
            data.extend([smiles])

            df_molecules.loc[len(df_molecules.index)] = data
            
    except:
        continue
        
df_molecules.to_csv('OPEP2_dataset_SMILES.csv')